In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np

RETRIEVE CATEGORIES

In [2]:
start_time = time.time()
driver = webdriver.Chrome()
driver.switch_to_window(driver.current_window_handle)
driver.maximize_window()
driver.get('https://www.happyfresh.my/stores/cold-storage-the-fresh-food-people/locations/535/')
# GET LINKS FOR EACH-SUB CATEGORY
outputs = []
category = driver.find_elements_by_class_name('subcategories')
for stage1 in category:
    main_cat = stage1.find_elements_by_tag_name('span')[-1].get_attribute('innerText')
    sub_list = stage1.find_elements_by_tag_name('a')
    for stage2 in sub_list[1:]:
        sub_cat = stage2.get_attribute('innerText')
        sub_url = stage2.get_attribute('href')
        outputs.append([main_cat, sub_cat, sub_url])
time.sleep(1)
driver.quit()
print('*** Process Completeed ***')
print('Time used:\t',time.time()-start_time)

data = pd.DataFrame(outputs, columns=['main_cat', 'sub_cat', 'sub_url'])
wanted = ['Monthly Specials','Fresh Produce','Meat & Seafood','Dairy & Eggs','Beverages','Frozen','Deli','Organic Products',\
          'Non-halal','Alcohol & Tobacco','Other Wines Selection']
data = data[data.main_cat.isin(wanted)]
print('Dataframe shape:\t',data.shape)
data.to_csv('happyfresh_cold_storage.csv',index=False)
print(data.main_cat.unique())

*** Process Completeed ***
Time used:	 17.22148561477661
Dataframe shape:	 (62, 3)
['Monthly Specials' 'Fresh Produce' 'Meat & Seafood' 'Dairy & Eggs'
 'Beverages' 'Frozen' 'Deli' 'Organic Products' 'Non-halal'
 'Alcohol & Tobacco' 'Other Wines Selection']


In [3]:
data = pd.read_csv('happyfresh_cold_storage.csv')

In [4]:
data.head()

,main_cat,sub_cat,sub_url
0,Monthly Specials,Value To Buy,https://www.happyfresh.my/stores/cold-storage-...
1,Fresh Produce,Fresh Fruits,https://www.happyfresh.my/stores/cold-storage-...
2,Fresh Produce,Fresh Vegetables,https://www.happyfresh.my/stores/cold-storage-...
3,Fresh Produce,Fresh Herbs,https://www.happyfresh.my/stores/cold-storage-...
4,Fresh Produce,Packaged Fruit & Vegetables,https://www.happyfresh.my/stores/cold-storage-...


RETRIEVE ITEMS

In [51]:
start_time = time.time()
driver = webdriver.Chrome()
driver.switch_to_window(driver.current_window_handle)
driver.maximize_window()
driver.get('https://www.happyfresh.my/stores/cold-storage-the-fresh-food-people/locations/535/')
item_datas = []
for i, row in data.iterrows():
    driver.get(row.sub_url)
    for n in range(18):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
    item_list = driver.find_elements_by_class_name('product-summary')
    for item in item_list:
        promo = item.find_element_by_class_name('promotion-container').get_attribute('innerText').split('\n')[0]
        older_price = item.find_element_by_class_name('previous-price-container').get_attribute('innerText').split('\n')[0][2:]
        if older_price == '':
            older_price = '0.00'
        item_price = item.find_element_by_tag_name('strong').get_attribute('innerText')[2:]
        item_name = item.find_element_by_class_name('product-name').get_attribute('innerText')
        item_unit = item.find_element_by_class_name('product-unit').get_attribute('innerText')
        item_unit_price = item.find_element_by_class_name('product-unit-price').get_attribute('innerText')
        item_datas.append([row.main_cat, row.sub_cat, item_name, item_unit, item_unit_price, item_price, older_price, promo])
time.sleep(1)
driver.quit()
print('*** Process Completeed ***')
print('Time used:\t',time.time()-start_time)

*** Process Completeed ***
Time used:	 1873.824696779251


In [15]:
item_df = pd.DataFrame(item_datas, columns=['main_cat','sub_cat','item_name','item_unit','item_unit_price','item_price',\
                                            'item_old_price','promotion'])
print('Dataframe shape:\t',item_df.shape)
item_df.to_csv('cold_storage_database.csv',index=False)
item_df.head()

Dataframe shape:	 (2130, 8)


,main_cat,sub_cat,item_name,item_unit,item_unit_price,item_price,item_old_price,promotion
0,Monthly Specials,Value To Buy,Arla Natural Cheese Sliced Bundle,1 bundle,RM22.40,22.40,28.00,20% Off
1,Monthly Specials,Value To Buy,Arla Triangles 8 Portion Cheese + Arla Natural...,1 bundle,RM13.16,13.16,16.45,20% Off
2,Monthly Specials,Value To Buy,Arla Triangles 8 Portion Cheese Bundle,1 bundle,RM11.92,11.92,14.90,20% Off
3,Monthly Specials,Value To Buy,Arla Triangles 8 Portion Cheese + Gardenia...,1 bundle,RM8.50,8.50,9.99,15% Off
4,Monthly Specials,Value To Buy,Arla Natural Cheese Block Bundle,1 bundle,RM23.92,23.92,29.90,20% Off
